In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

%matplotlib inline

# Test Setini Tahmin Etme ve Kaggle'a sonuçları yükleme

In [2]:
# Eğitim ve test veri setleri dosya olarak ayrılmış. Bizden test veri seti sonuçlarını Kaggle'a yüklememiz bekleniyor.
# Bunun için modelimizi buradaki tüm veri ile (X) eğitip test.csv dosyasındaki Survived bilgisi olmayan test seti sonuçlarını
# tahmin (predict()) edip diske yazıp Kaggle'a yüklememiz gerekir.

In [3]:
# veri kaynağı: https://www.kaggle.com/c/titanic
df_test = pd.read_csv("D:\\OneDrive\\egitim\\verilen\\bogazici-compec-python-ml\\Datasets\\titanic\\test.csv")

In [4]:
df_test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [5]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
PassengerId    418 non-null int64
Pclass         418 non-null int64
Name           418 non-null object
Sex            418 non-null object
Age            332 non-null float64
SibSp          418 non-null int64
Parch          418 non-null int64
Ticket         418 non-null object
Fare           417 non-null float64
Cabin          91 non-null object
Embarked       418 non-null object
dtypes: float64(2), int64(4), object(5)
memory usage: 36.0+ KB


In [6]:
# Age, Fare, ve Cabin niteliklerinde boşluklar var.

# Age için boşluk doldurma

In [7]:
# Age boşluklarını ortanca ile doldurmak için aşağıdaki kütüpheneyi kullanıyoruz
from sklearn.impute import SimpleImputer
# Boş bir SimpleImputer nesnesi yaratalım
imputer = SimpleImputer(strategy='median')
# SimpleImputer nesnesini Age ile eğitip hesaplanan ortancayı boşluklara dlduralım ve Age niteliğine atayalım
df_test['Age'] = imputer.fit_transform(df_test['Age'].values.reshape(-1,1))

# Fare için boşluk doldurma

In [8]:
df_test.groupby(['Pclass']).Fare.agg(['mean']).head()

,mean
Pclass,
1,94.280297
2,22.202104
3,12.459678


## Hangi yolcuya ait Fare bilgisi eksik?

In [9]:
df_test[pd.isnull(df_test['Fare'])].head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
152,1044,3,"Storey, Mr. Thomas",male,60.5,0,0,3701,NaN,NaN,S


In [10]:
# Boş değeri Pclass ortalaması değeri ile dolduralım. Çünkü yukarıda gördüğümüz gibi bilet fiyatları Pclass'a duyarlı.
df_test.iloc[152,8] = 12.45

In [11]:
df_test[df_test.index == 152].head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
152,1044,3,"Storey, Mr. Thomas",male,60.5,0,0,3701,12.45,NaN,S


In [12]:
# Boşluk dolmuş

# Cabin niteliğini düşürme

In [13]:
df_test.drop('Cabin', axis=1, inplace=True)

In [14]:
df_test.head(2)

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,S


In [15]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 10 columns):
PassengerId    418 non-null int64
Pclass         418 non-null int64
Name           418 non-null object
Sex            418 non-null object
Age            418 non-null float64
SibSp          418 non-null int64
Parch          418 non-null int64
Ticket         418 non-null object
Fare           418 non-null float64
Embarked       418 non-null object
dtypes: float64(2), int64(4), object(4)
memory usage: 32.7+ KB


In [16]:
# Tüm boşluklar tamam.

# Veriyi X,y olarak bölme (nitelikler matrisi ve hedef değişken - dikey bölme)¶

In [17]:
# Burada y niteliği olmadığı için bölme yapmıyoruz. Hepsi X olacak. Sadece sütun sıralamasını değiştirelim

In [18]:
df_test = df_test[['PassengerId', 'Name', 'Ticket', 'Embarked','Pclass', 'Sex', 'Age', 'SibSp',
       'Parch',  'Fare']]
df_test.head()

,PassengerId,Name,Ticket,Embarked,Pclass,Sex,Age,SibSp,Parch,Fare
0,892,"Kelly, Mr. James",330911,Q,3,male,34.5,0,0,7.8292
1,893,"Wilkes, Mrs. James (Ellen Needs)",363272,S,3,female,47.0,1,0,7.0000
2,894,"Myles, Mr. Thomas Francis",240276,Q,2,male,62.0,0,0,9.6875
3,895,"Wirz, Mr. Albert",315154,S,3,male,27.0,0,0,8.6625
4,896,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",3101298,S,3,female,22.0,1,1,12.2875


In [19]:
# 'Embarked','Pclass', 'Sex', 'Age', 'SibSp', 'Parch',  'Fare'
X = df_test.iloc[:,3:].values
X.shape

(418, 7)

In [20]:
X[:10]

array([['Q', 3, 'male', 34.5, 0, 0, 7.8292],
       ['S', 3, 'female', 47.0, 1, 0, 7.0],
       ['Q', 2, 'male', 62.0, 0, 0, 9.6875],
       ['S', 3, 'male', 27.0, 0, 0, 8.6625],
       ['S', 3, 'female', 22.0, 1, 1, 12.2875],
       ['S', 3, 'male', 14.0, 0, 0, 9.225],
       ['Q', 3, 'female', 30.0, 0, 0, 7.6292],
       ['S', 2, 'male', 26.0, 1, 1, 29.0],
       ['C', 3, 'female', 18.0, 0, 0, 7.2292],
       ['S', 3, 'male', 21.0, 2, 0, 24.15]], dtype=object)

# Kategorik Nitelik Dönüştürme

In [21]:
# kategorik nitelikler: 'Embarked','Pclass', 'Sex' Indeksleri: 
from sklearn.preprocessing import LabelEncoder
labelencoder_embarked_test = LabelEncoder()
labelencoder_sex_test = LabelEncoder()
# Pclass zaten nümerik kodlandığı için onu burada dönüştürmeme gerek yok

In [22]:
X[:,0] = labelencoder_embarked_test.fit_transform(X[:,0])
X[:,2] = labelencoder_sex_test.fit_transform(X[:,2])

In [23]:
X[:10]

array([[1, 3, 1, 34.5, 0, 0, 7.8292],
       [2, 3, 0, 47.0, 1, 0, 7.0],
       [1, 2, 1, 62.0, 0, 0, 9.6875],
       [2, 3, 1, 27.0, 0, 0, 8.6625],
       [2, 3, 0, 22.0, 1, 1, 12.2875],
       [2, 3, 1, 14.0, 0, 0, 9.225],
       [1, 3, 0, 30.0, 0, 0, 7.6292],
       [2, 2, 1, 26.0, 1, 1, 29.0],
       [0, 3, 0, 18.0, 0, 0, 7.2292],
       [2, 3, 1, 21.0, 2, 0, 24.15]], dtype=object)

# One Hot Encoder - ColumnTransformer¶

In [24]:
from sklearn.preprocessing import OneHotEncoder
onehotencoder = OneHotEncoder(handle_unknown='ignore', categories='auto')

In [25]:
# Birden sütunu ilgilendiren dönüşümleri kolay yapabilmek için kullanışlı bir sınıf (Estimator kökenli)
# Biz burada sadece OneHotEncoder için kullanacağız.
from sklearn.compose import ColumnTransformer

In [26]:
# one_hot_encode_with_ct bizim kendi verdiğiiz isimdir. onehotencoder'ı yukarıda yarattık. [0,1,2] ise dönüşecek
# niteliklerin indeks değerleridir.
ct = ColumnTransformer(
    [('one_hot_encode_with_ct', onehotencoder, [0,1,2]),], 
    remainder='passthrough'
)

In [27]:
# ColumnTransformer nesnesi ile onehotencoder nesnesini eğitip gerekli dönüşümleri gerçekleştirip X'e tekrar atayalım.
# Burada X'in boyutu(shape) değişecektir.
X = ct.fit_transform(X)

In [28]:
X[:10]

array([[0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 34.5, 0, 0, 7.8292],
       [0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 47.0, 1, 0, 7.0],
       [0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 62.0, 0, 0, 9.6875],
       [0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 27.0, 0, 0, 8.6625],
       [0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 22.0, 1, 1, 12.2875],
       [0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 14.0, 0, 0, 9.225],
       [0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 30.0, 0, 0, 7.6292],
       [0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 26.0, 1, 1, 29.0],
       [1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 18.0, 0, 0, 7.2292],
       [0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 21.0, 2, 0, 24.15]],
      dtype=object)

In [29]:
# Boyut değişmiş olmalı. Çnkü gölge değişkenler eklendi
X.shape

(418, 12)

In [30]:
# (889, 7) idi Embarked için ['C' 'Q' 'S'] ve Sex için ['female' 'male']
# toplam 5 adetgölge değişkenler eklendi ve boyut (889, 12) oldu.

# Feature Scaling (Standardizasyon)

In [31]:
from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler()
X = sc_X.fit_transform(X)

D:\Anaconda\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
D:\Anaconda\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


# Modeli okuma

In [32]:
from sklearn.externals import joblib
classifier = joblib.load("07.classification_with_titanic_train_data.pkl")

# Predictions

In [33]:
y_pred = classifier.predict(X)

In [34]:
type(y_pred)

numpy.ndarray

# Tahmin sonuçlarını dataframe'e ekleme

In [35]:
df_test['Survived'] = y_pred

In [36]:
df_test.head()

,PassengerId,Name,Ticket,Embarked,Pclass,Sex,Age,SibSp,Parch,Fare,Survived
0,892,"Kelly, Mr. James",330911,Q,3,male,34.5,0,0,7.8292,0
1,893,"Wilkes, Mrs. James (Ellen Needs)",363272,S,3,female,47.0,1,0,7.0000,0
2,894,"Myles, Mr. Thomas Francis",240276,Q,2,male,62.0,0,0,9.6875,0
3,895,"Wirz, Mr. Albert",315154,S,3,male,27.0,0,0,8.6625,0
4,896,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",3101298,S,3,female,22.0,1,1,12.2875,1


# Kaggle'a yüklenecek nitelikler ile yeni df oluşturma

In [37]:
submission_for_kaggle = df_test[['PassengerId','Survived']]

In [38]:
submission_for_kaggle.head()

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1


# Kaggle'a yüklenecek df'i diske yazma

In [39]:
submission_for_kaggle.to_csv(path_or_buf="submission_for_kaggle.csv", 
          sep=",", index=False, header=True)

In [40]:
# Kontrol
submission_for_kaggle_reread = pd.read_csv("submission_for_kaggle.csv")
submission_for_kaggle_reread.head()

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1
